In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Robust_scaller_result.csv',encoding='cp949')

In [3]:

from sklearn.model_selection import train_test_split

In [4]:
a = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]
act = data['ACTCD']

In [5]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT',  'KOSPI', '환율', 'GDP', 
       '국고채', '유가등락률',
       'KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]


In [6]:
act= act-1
act= act*-1

In [7]:

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 34) (11736, 34) (27381,) (11736,)


In [8]:
'''
1. ENN
2. ADASYN
3. BORDERLINE SMOTE
4. Tomeklinks
5 SOTEENN
6.SMOTETOMEK
'''
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()
accuracies = []
matrix_kd_tree_uniform_BOST = []

In [9]:
keys_list

dict_keys(['enn', 'ada', 'smte', 'smtt', 'tk', 'sm'])

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix, classification_report


In [11]:
for i in keys_list:
    model = RandomForestClassifier(n_estimators=200,max_depth=50)
    model.fit(data_sets[i][0], data_sets[i][1])
                            
                    # 결정계수
    relation_square = model.score(data_sets[i][0], data_sets[i][1])
    print('결정계수 : ', relation_square)
                    
                    # 예측
    pred_y = model.predict(test_x)
    pr = precision_score(y_true=test_y,y_pred=pred_y)
    r = recall_score(y_true=test_y,y_pred=pred_y)
    f1 = f1_score(y_true=test_y,y_pred=pred_y)
    rca = roc_auc_score(y_true=test_y,y_score=pred_y)
    cfm = confusion_matrix(y_true=test_y,y_pred=pred_y)
    matrix_kd_tree_uniform_BOST.append((i,pr,r,f1,rca,model.score(test_x, test_y),cfm))
                # 정확도 확인
    print(i)
    print('정확도 : ', metrics.accuracy_score(test_y, pred_y))
    print(classification_report(test_y, pred_y, target_names=['사고보증', '정상보증']))
    print('\n')

결정계수 :  0.9991998933191092
enn
정확도 :  0.9239945466939332
              precision    recall  f1-score   support

        사고보증       0.72      0.27      0.40      1070
        정상보증       0.93      0.99      0.96     10666

    accuracy                           0.92     11736
   macro avg       0.82      0.63      0.68     11736
weighted avg       0.91      0.92      0.91     11736



결정계수 :  0.9988332796910203
ada
정확도 :  0.9133435582822086
              precision    recall  f1-score   support

        사고보증       0.55      0.29      0.38      1070
        정상보증       0.93      0.98      0.95     10666

    accuracy                           0.91     11736
   macro avg       0.74      0.63      0.66     11736
weighted avg       0.90      0.91      0.90     11736



결정계수 :  1.0
smte
정확도 :  0.8446659850034083
              precision    recall  f1-score   support

        사고보증       0.29      0.47      0.36      1070
        정상보증       0.94      0.88      0.91     10666

    accuracy         

In [12]:
matrix_kd_tree_uniform_BOST

[('enn',
  0.9314850785802579,
  0.9891243202700168,
  0.9594397962895598,
  0.6319453377051019,
  0.9239945466939332,
  array([[  294,   776],
         [  116, 10550]], dtype=int64)),
 ('ada',
  0.9316453431153261,
  0.9762797674854679,
  0.9534404614750721,
  0.6311305379483414,
  0.9133435582822086,
  array([[  306,   764],
         [  253, 10413]], dtype=int64)),
 ('smte',
  0.9436139259556536,
  0.8817738608663042,
  0.9116463917026122,
  0.6782701080032456,
  0.8446659850034083,
  array([[ 508,  562],
         [1261, 9405]], dtype=int64)),
 ('smtt',
  0.9319018956068638,
  0.9725295330958185,
  0.9517823553700051,
  0.6320591590712736,
  0.9104464894342195,
  array([[  312,   758],
         [  293, 10373]], dtype=int64)),
 ('tk',
  0.9282906804087693,
  0.9964372773298331,
  0.9611575853493104,
  0.6145737788518324,
  0.9268064076346285,
  array([[  249,   821],
         [   38, 10628]], dtype=int64)),
 ('sm',
  0.9325792261423826,
  0.973935870991937,
  0.952808988764045,
  0.63

In [30]:
type(test_y)

pandas.core.series.Series

In [32]:
aaa= test_y.to_frame(name='act')

In [42]:
type(aaa)

pandas.core.frame.DataFrame

In [21]:
ab = aaa[aaa['act']==1]

NameError: name 'aaa' is not defined

In [46]:
ab

,act
8618,1
10479,1
23298,1
20234,1
36829,1
...,...
34635,1
7453,1
1610,1
20619,1
